## Load Libraries

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

# distributions
import scipy.integrate as integrate
from scipy.integrate import quad, quad_vec
from scipy.stats import invgamma
from scipy.stats import levy
from scipy.special import gamma, erfinv

import time
from datetime import datetime, timedelta

import sys
import importlib # ref: https://askubuntu.com/questions/144698/find-a-file-by-name-using-command-line
import seaborn as sns

import os

In [2]:
sys.path.append('../tools')
import tools

In [3]:
importlib.reload(tools)

<module 'tools' from '../tools/tools.py'>

## Download Data

In [4]:
# download and store data in a file
dfs = []
for file in os.listdir('data/2017'):
    if '.csv' in file:
        df = pd.read_csv(f'data/2017/{file}',  usecols = ['date', 'ts'])
        dfs.append(df)


In [5]:
# concatenate the dataframe into one dataframe
df = pd.concat(dfs)


In [6]:
#df_sample = pd.read_csv('data/2017/taqAAPL2017-12-29.csv', usecols = ['date', 'ts'])

## Cleaning Data

### Removing Duplicate Times

In [7]:
# create date time column
df['date_ts'] = pd.to_datetime(df['date'] + ' '+ df['ts'], format = '%Y-%M-%D %H:%M:%S.%f', infer_datetime_format = True)

# sort the data according to time
df = df.sort_values(by = 'date_ts')

In [8]:
df = df.drop_duplicates(subset = 'date_ts', keep = 'first')


### Taking differences and removing unnecessary rows

In [9]:
# calculate the difference in time
df['delta_ts'] = df['date_ts'].diff().dt.total_seconds()

In [10]:
# remove the first observation of each day
# we can see when the time difference between trading is big, they are on different dates
print(sum(df['delta_ts']>1000))
print(len(df[df['delta_ts']>1000].date.unique()))

250
250


In [11]:
# drop unnecessayr values
df_cleaned = df[~(df['delta_ts']>1000)].copy()

# reset index
df_cleaned = df_cleaned.reset_index(drop=True)

In [12]:
len(df) - len(df_cleaned)

250

In [13]:
# remove first observations where delta_ts is nan
df_cleaned.head()

,date,ts,date_ts,delta_ts
0,2017-01-03,09:30:00.001271,2017-01-03 09:30:00.001271,NaN
1,2017-01-03,09:30:00.046605,2017-01-03 09:30:00.046605,0.045334
2,2017-01-03,09:30:00.090184,2017-01-03 09:30:00.090184,0.043579
3,2017-01-03,09:30:00.090207,2017-01-03 09:30:00.090207,0.000023
4,2017-01-03,09:30:00.090214,2017-01-03 09:30:00.090214,0.000007


In [14]:
# check if trade on first trades on each day is removed
unique_dates = df_cleaned['date'].unique()

In [73]:
for date in unique_dates[0:10]:
    
    max_ts = max(df_cleaned[df_cleaned['date']== date]['ts'])
    
    last_trade_index = df_cleaned.loc[(df_cleaned['date']==date)& (df_cleaned['ts'] == max_ts), ].index
    
    print(df_cleaned.iloc[last_trade_index[0]: last_trade_index[0]+2, ])
    
    
    

              date               ts                    date_ts  delta_ts
138456  2017-01-03  15:59:59.998930 2017-01-03 15:59:59.998930  0.000022
1       2017-01-04  09:30:00.007337 2017-01-04 09:30:00.007337  0.000004
              date               ts                    date_ts  delta_ts
113631  2017-01-03  15:14:59.771501 2017-01-03 15:14:59.771501  0.000061
113632  2017-01-03  15:15:00.033733 2017-01-03 15:15:00.033733  0.262232
              date               ts                    date_ts  delta_ts
120469  2017-01-03  15:37:52.862482 2017-01-03 15:37:52.862482  0.025198
120470  2017-01-03  15:37:53.683727 2017-01-03 15:37:53.683727  0.821245
             date               ts                    date_ts  delta_ts
15467  2017-01-04  09:51:05.558356 2017-01-04 09:51:05.558356  0.000009
15468  2017-01-04  09:51:05.558744 2017-01-04 09:51:05.558744  0.000388
             date               ts                    date_ts  delta_ts
25077  2017-01-04  10:16:19.304382 2017-01-04 10:16:19.

### Assign bins

In [15]:
# assign bins
# create bins of 30 minutes
market_open = datetime.strptime('09:30', '%H:%M')

bins = [market_open.strftime("%H:%M")]

for x in range(13):
    market_open = market_open + timedelta(minutes = 30)
    bins.append(market_open.strftime("%H:%M"))



In [16]:
# define a function to assing bin number to data
def assign_bin(time_string, bins):
    
    bin_numb = 0
    prev_bin = '9:30'
    
    for bin_i in bins:
        if (time_string >= prev_bin) &  (time_string < bin_i):  
            return bin_numb
        
        bin_numb += 1
        prev_bin = bin_i

In [17]:
# assing bins
df_cleaned.loc[:, 'bin'] = df_cleaned['ts'].apply(lambda x: assign_bin(x, bins))

In [18]:
# check maximum value for the duration
np.nanmax(df_cleaned['delta_ts'])

708.5275730000001

In [19]:
df_cleaned[df_cleaned['delta_ts']==708.5275730000001]

,date,ts,date_ts,delta_ts,bin
17633651,2017-07-03,15:17:39.804169,2017-07-03 15:17:39.804169,708.527573,12


In [20]:
df_cleaned.iloc[17633650:17633652, ]

,date,ts,date_ts,delta_ts,bin
17633650,2017-07-03,15:05:51.276596,2017-07-03 15:05:51.276596,281.229521,12
17633651,2017-07-03,15:17:39.804169,2017-07-03 15:17:39.804169,708.527573,12


## Export Cleaned Data

In [21]:
df_cleaned.to_csv(r'data/apple_cleaned.csv', index=False)